In [6]:
import networkx as nx
import numpy as np
import matplotlib.pyplot as plt
import random
import time
from neo4j import GraphDatabase

class UAVNetworkSimulation:

    def __init__(self, uri, auth):
        self.driver = GraphDatabase.driver(uri, auth=auth)

    def close(self):
        self.driver.close()
    
    def create_initial_graph(self, num_uavs, connection_range, ground_station_pos, backbone_range):
        G = nx.Graph()
        typical_throughput = (50, 100)
        typical_latency = (10, 50)

        for i in range(num_uavs):
            throughput = random.randint(*typical_throughput)
            latency = random.randint(*typical_latency)
            G.add_node(i, pos=(random.randint(0, 100), random.randint(0, 100)),
                       throughput=throughput, latency=latency)

        backbone_uav_id = num_uavs
        G.add_node(backbone_uav_id, pos=(random.randint(0, 100), random.randint(0, 100)), is_backbone=True,
                   throughput=throughput, latency=latency)

        for i in range(num_uavs):
            pos_i = G.nodes[i]['pos']
            pos_backbone = G.nodes[backbone_uav_id]['pos']
            distance = np.linalg.norm(np.array(pos_i) - np.array(pos_backbone))
            if distance <= backbone_range:
                G.add_edge(i, backbone_uav_id, weight=distance)

        ground_station_id = num_uavs + 1
        G.add_node(ground_station_id, pos=ground_station_pos, is_ground_station=True, throughput=throughput, latency=latency)
        G.add_edge(backbone_uav_id, ground_station_id)

        for i in range(num_uavs):
            for j in range(i + 1, num_uavs):
                pos_i = G.nodes[i]['pos']
                pos_j = G.nodes[j]['pos']
                distance = np.linalg.norm(np.array(pos_i) - np.array(pos_j))
                if distance <= connection_range:
                    G.add_edge(i, j, weight=distance)

        return G

    def add_attack_node(self, G, target_ids, num_uavs, attack_node_pos=(random.randint(0, 100), random.randint(0, 100))):
        attack_node_id = max(G.nodes()) + 1
        G.add_node(attack_node_id, pos=attack_node_pos, nodeName="Attack Node", is_attack_node=True, throughput=1000, latency=1)
        for target_id in target_ids:
            if target_id in G.nodes():
                G.add_edge(attack_node_id, target_id, weight=5)
        return attack_node_id

    def simulate_ddos_traffic(self, G, attack_node_id, target_ids, num_packets, packet_size):
        packets = []
        for target_id in target_ids:
            for _ in range(num_packets):
                packets.append({
                    'source': attack_node_id,
                    'target': target_id,
                    'size': packet_size,
                    'is_attack': True
                })
        return packets

    def route_packets(self, G, packets):
        routed_packets = []
        for packet in packets:
            if nx.has_path(G, packet['source'], packet['target']):
                path = nx.shortest_path(G, packet['source'], packet['target'])
                packet['path'] = path
                packet['delivered'] = True
                packet['total_latency'] = sum(G.nodes[node]['latency'] for node in path)
            else:
                packet['delivered'] = False
                packet['total_latency'] = None
            routed_packets.append(packet)
        return routed_packets

    def process_simulation_results(self, routed_packets):
        total_latency = sum(packet['total_latency'] for packet in routed_packets if packet['delivered'])
        successful_deliveries = sum(1 for packet in routed_packets if packet['delivered'])
        average_latency = total_latency / successful_deliveries if successful_deliveries > 0 else 0
        print(f"Average Latency: {average_latency} ms")

    def run_simulation(self, G, total_time, update_interval, attack_node_id, target_ids, num_packets, packet_size):
        start_time = time.time()
        while time.time() - start_time < total_time:
            ddos_packets = self.simulate_ddos_traffic(G, attack_node_id, target_ids, num_packets, packet_size)
            routed_packets = self.route_packets(G, ddos_packets)
            self.process_simulation_results(routed_packets)
            time.sleep(update_interval)

# Example of initializing and running the simulation
simulation = UAVNetworkSimulation("neo4j://localhost:7687", ("neo4j", "password"))
num_uavs = 10
connection_range = 30
ground_station_pos = (50, 50)
backbone_range = 50
uav_network = simulation.create_initial_graph(num_uavs, connection_range, ground_station_pos, backbone_range)
attack_node_id = simulation.add_attack_node(uav_network, [0, 1], num_uavs)
simulation.run_simulation(uav_network, 60, 5, attack_node_id, [0, 1], 1000, 10)
simulation.close()


Average Latency: 28.0 ms
Average Latency: 28.0 ms
Average Latency: 28.0 ms
Average Latency: 28.0 ms
Average Latency: 28.0 ms
Average Latency: 28.0 ms
Average Latency: 28.0 ms
Average Latency: 28.0 ms
Average Latency: 28.0 ms
Average Latency: 28.0 ms
Average Latency: 28.0 ms
Average Latency: 28.0 ms
